In [1]:
import os
import json
import tqdm
import yaml
import pandas as pd

import torch
import numpy as np
import matplotlib.pyplot as plt

from omegaconf import OmegaConf
from pathlib import Path
from tqdm.auto import tqdm

/home/denis/miniconda3/envs/uncertainty_evaluation/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%%bash
mkdir -p results
fusermount -uz results
sshfs compute.ies:/mnt/work/deep_al/results/udal/hparams results
ls results

CIFAR10


In [3]:
def load_json(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    return data

def load_results(path):
    path = Path(path)
    assert path.is_dir(), 'Path does not exist.'
        
    results = {}
    exp_json = path / 'results.json'
    exp_cfg = path / '.hydra' / 'config.yaml'
    try:
        cfg =  OmegaConf.load(exp_cfg)
        data = load_json(exp_json)
    except:
        print(f'{path} has missing results.')
        return

    results['cfg'] = cfg
    results['results'] = data
    return results


In [4]:
results = {}
root_path = Path('results/CIFAR10/resnet18/budget2000/')
experiment_paths = root_path.glob('*')
for exp_path in experiment_paths:
    exp_results = []
    for seed_exp_path in exp_path.glob('seed*'):
        exp_results.append(load_results(seed_exp_path))
    results[exp_path.name] = exp_results

exp_metrics = {}
for exp_name in results:
    learning_rates = []
    val_accs = []
    val_nlls = []
    val_tces = []

    exp_results = results[exp_name]
    for seed_results in exp_results:
        train_history = seed_results['results']['train_history']
        val_stats = seed_results['results']['val_stats']

        val_accs.append(val_stats['test_acc1'])
        val_nlls.append(val_stats['test_nll'])
        val_tces.append(val_stats['test_tce'])
    weight_decay = seed_results['cfg']['model']['optimizer']['weight_decay']
    learning_rate = seed_results['cfg']['model']['optimizer']['lr']
    
    exp_metrics[exp_name] = {
        'lr': learning_rate, 'weight_decay': weight_decay,
        'Val Acc': np.mean(val_accs), 'Val Acc std':  np.std(val_accs),
        'Val NLL': np.mean(val_nlls), 'Val NLL std':  np.std(val_nlls),
        'Val TCE': np.mean(val_tces), 'Val TCE std':  np.std(val_tces),
    }

df = pd.DataFrame(exp_metrics).T
print(df.to_markdown())

|                   |    lr |   weight_decay |   Val Acc |   Val Acc std |   Val NLL |   Val NLL std |   Val TCE |   Val TCE std |
|:------------------|------:|---------------:|----------:|--------------:|----------:|--------------:|----------:|--------------:|
| lr0.001__wd0.0005 | 0.001 |         0.0005 |   56.52   |      1.34353  |  2.07962  |     0.0811032 | 0.289272  |    0.0138105  |
| lr0.001__wd0.05   | 0.001 |         0.05   |   58.96   |      0.765768 |  1.35726  |     0.0164111 | 0.163177  |    0.00801945 |
| lr0.01__wd0.05    | 0.01  |         0.05   |   74.9533 |      0.550715 |  0.85128  |     0.0123957 | 0.0714206 |    0.00193299 |
| lr0.01__wd0.0005  | 0.01  |         0.0005 |   62.1    |      1.07443  |  1.99491  |     0.0320275 | 0.273112  |    0.00733282 |
| lr0.1__wd0.005    | 0.1   |         0.005  |   72.2    |      1.0215   |  0.915496 |     0.0223846 | 0.0873935 |    0.00953427 |
| lr0.01__wd0.005   | 0.01  |         0.005  |   64.5667 |      0.324277 |  1.27185

In [5]:
results = {}
root_path = Path('results/CIFAR10/resnet18/budget20000/')
experiment_paths = root_path.glob('*')
for exp_path in experiment_paths:
    exp_results = []
    for seed_exp_path in exp_path.glob('seed*'):
        exp_results.append(load_results(seed_exp_path))
    results[exp_path.name] = exp_results

exp_metrics = {}
for exp_name in results:
    learning_rates = []
    val_accs = []
    val_nlls = []
    val_tces = []

    exp_results = results[exp_name]
    for seed_results in exp_results:
        train_history = seed_results['results']['train_history']
        val_stats = seed_results['results']['val_stats']

        val_accs.append(val_stats['test_acc1'])
        val_nlls.append(val_stats['test_nll'])
        val_tces.append(val_stats['test_tce'])
    weight_decay = seed_results['cfg']['model']['optimizer']['weight_decay']
    learning_rate = seed_results['cfg']['model']['optimizer']['lr']
    
    exp_metrics[exp_name] = {
        'lr': learning_rate, 'weight_decay': weight_decay,
        'Val Acc': np.mean(val_accs), 'Val Acc std':  np.std(val_accs),
        'Val NLL': np.mean(val_nlls), 'Val NLL std':  np.std(val_nlls),
        'Val TCE': np.mean(val_tces), 'Val TCE std':  np.std(val_tces),
    }

df = pd.DataFrame(exp_metrics).T
print(df.to_markdown())

|                   |    lr |   weight_decay |   Val Acc |   Val Acc std |   Val NLL |   Val NLL std |   Val TCE |   Val TCE std |
|:------------------|------:|---------------:|----------:|--------------:|----------:|--------------:|----------:|--------------:|
| lr0.01__wd0.05    | 0.01  |         0.05   |  89.76    |      0.631822 |  0.396143 |   0.0176271   | 0.0662947 |    0.0022784  |
| lr0.1__wd0.0005   | 0.1   |         0.0005 |  91.8333  |      0.344224 |  0.305023 |   0.0201698   | 0.0424353 |    0.0024983  |
| lr0.01__wd0.005   | 0.01  |         0.005  |  92.3133  |      0.627445 |  0.270801 |   0.0195133   | 0.0253762 |    0.00352964 |
| lr0.001__wd0.0005 | 0.001 |         0.0005 |  85.9267  |      0.122566 |  0.647878 |   0.0194588   | 0.107333  |    0.00199614 |
| lr0.001__wd0.05   | 0.001 |         0.05   |  91.76    |      0.31027  |  0.350146 |   0.011713    | 0.0757428 |    0.00166664 |
| lr0.01__wd0.0005  | 0.01  |         0.0005 |  90.2467  |      0.482309 |  0.37441